# Régression logistique

La régression logistique, aussi appelée __modèle logit__ ou __régression logit__, est une régression où la variable dépendante est de type **catégoriel**.

Par souci de simplification, nous allons regarder ici uniquement la régression logistique binomiale avec 2 catégories (classes) pour la variable dépendante. Ce sont les modèles de régression logistique multinomial qui couvrent les cas de plus de deux catégories.

La régression logistique peut être appliquée à de nombreux problèmes de classification:

- Sur la base du nombre d'heures travaillées, est-ce qu'un étudiant va-t-il réussir ses études ou non?
- Sur la base du contenu d'un e-mail, est-ce qu'il s'agit du spam ou non?
- Étant donné les caractéristiques d'un paiement par carte de crédit, est-ce qu'il s'agit d'une fraude ou non?
- Étant donné les caractéristiques démographiques, est-ce que la commune est rurale ou urbaine?

et pour les problèmes multinominaux:

- Pour quel parti ou candidat une personne va voter?
- Quelle est la typologie d'une commune (urbain, péri-urbain, rural, etc.)?
- ...

De manière générale, la régression logistique dans sa version binomiale peut être utilisée pour faire des prédictions binaires (oui/non, 1/0, juste/faux, etc.).

En gros, la régression logistique binomiale essaie d'évaluer la probabilité pour l'une ou l'autre des classes. Si nous prenons l'example de la relation entre le nombre d'heures d'étude pour un examen et la probabilité de réussite à l'examen, nous pouvons obtenir une courbe qui pourrait ressembler à:

![](https://upload.wikimedia.org/wikipedia/commons/6/6d/Exam_pass_logistic_curve.jpeg)  
*Source: https://upload.wikimedia.org/wikipedia/commons/6/6d/Exam_pass_logistic_curve.jpeg*

Dans ce cas, la réussite de l'examen (catégories: réussite et échec) est la variable dépendante, et le nombre d'heures d'étude la seule variable indépendante. Évidemment, on pourrait ajouter d'autres variables indépendantes...



Pour montrer comment on calcule une régression logistique dans R, nous allons regarder à nouveau les régions Z05. Cette fois, nous essayons de déterminer si une région fait partie d'un centre urbain ou non, sur la base de la typologie territoriale de l'OFS ([Centres urbains et espace sous influence des centres urbains au 18.12.2014](https://www.bfs.admin.ch/bfs/fr/home/statistiques/themes-transversaux/analyses-spatiales/niveaux-geographiques.assetdetail.478314.html)).

Cette typologie est basée sur les communes et contient 3 catégories:

- Centres urbains
- Espaces sous influence des centres urbains
- Espaces hors influence des centres urbains

Cette typologie a été appliquée aux régions Z05, et le nombre de catégories réduit à 2:
- Centres urbains (`type_binaire=U`)
- Autres espaces (`type_binaire=N`)

Nous essayons de prédire si une région est un centre urbain ou non sur la base de quelques variables socio-économiques. En l'occurence, nous avons sélectionné les variables suivantes:

- **ADO**: proportion de personnes actives occupées
- **NADHO**: proportion de femmes et hommes de ménage
- **NADRET**: proportion de personnes actives à la retraite
- **AD3PRIM**: proportion de personnes actives dans le secteur primaire
- **AD3SEC**: proportion de personnes actives dans le secteur secondaire

Le choix de ces variables est en grande partie aléatoire. Le choix des variables est un problème à part que nous pourrons discuter ultérieurement (c'est le problème connu sous le nom de ***feature selection***).

Le tableau de données avec ces variables ainsi que la typologie se trouve dans le fichier [ch-socioeco-typologie.xlsx](ch-socioeco-typologie.xlsx) et la version pour lecture avec R dans [ch-socioeco-typologie.tsv](ch-socioeco-typologie.tsv) (mais sans explication des variables).

Il faut encore noter que toutes les variables représentent des proportions et non des valeurs absolues, car nous essayons de déterminer la typologie sur la base de la structure socio-économique et non la population absolue des régions.

Pour faire notre régression logistique, commençons par lire le tableau des données:

In [1]:
d = read.csv(file="ch-socioeco-typologie.tsv", sep="\t")

Et puis maintenant nous pouvons calculer la régression logistique. La fonction à utiliser est **[`glm`](http://stat.ethz.ch/R-manual/R-patched/library/stats/html/glm.html)**, en indiquant une valeur appropriée pour l'argument `family`:

In [2]:
reglogit = glm(d$typologie_binaire ~ ADO + NADHO + NADRET + AD3PRIM + AD3SEC, family=binomial(link="logit"), data=d) 

et afficher le résultat de la régression:

In [3]:
summary(reglogit)


Call:
glm(formula = d$typologie_binaire ~ ADO + NADHO + NADRET + AD3PRIM + 
    AD3SEC, family = binomial(link = "logit"), data = d)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.5507  -0.3308   0.2571   0.5225   3.5423  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)   28.408      4.407   6.446 1.15e-10 ***
ADO          -25.315      4.487  -5.642 1.68e-08 ***
NADHO        -20.103      3.570  -5.630 1.80e-08 ***
NADRET         1.655      1.114   1.486    0.137    
AD3PRIM     -100.366      6.757 -14.853  < 2e-16 ***
AD3SEC        -1.423      1.096  -1.299    0.194    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1810.23  on 1364  degrees of freedom
Residual deviance:  933.84  on 1359  degrees of freedom
AIC: 945.84

Number of Fisher Scoring iterations: 6


Nous pouvons maintenant **interpréter le résultat** de la régression.

La section sur les coefficient correspond au résultat donné par une régression linéaire simple. On peut y voir que les variables `NADRET` et `AD3SEC` ne sont pas statistiquement significtifs (ou plus formellement, la probabilité de se tromper en disant que le coefficient est non nul n'est pas négligeable).

Ce sont donc essentiellement la proportion des actifs occupés, des femmes et hommes de ménage, et la proportion de personnes actives dans le secteur primaire qui permettent de différencier les centres urbains du reste.

Quant à la qualité du résultat de régression, nous constatons tout d'abord que nous n'avons pas de $R^2$ ou similaire. Ceci est dû à la façon de calculer qui est très différente pour une régression logistique. La qualité du modèle est indiquée avec l'AIC et la déviance:

```
Null deviance: 1810.23  on 1364  degrees of freedom
Residual deviance:  933.84  on 1359  degrees of freedom
AIC: 945.84
```

Tout d'abord, l'Akaike Information Criterion (AIC) devrait être aussi petit que possible. La même chose est valable pour la déviance. Cependant, ceci nous ne donne pas une informations très parlante sur la qualité de notre modèle.

Nous avons plusieurs possibilités pour évaluer plus en détail la qualité du modèle. La première méthode est le **McFadden $R^2$**, qui est similaire au $R^2$ que nous connaissons déjà, sans toutefois être comparable. Pour pouvoir calculer le McFadden $R^2$, nous avons besoin du package additionnel `pscl` que nous pouvons charger comme suit:

In [ ]:
library(pscl)

Si le package n'est pas installé, nous pouvons l'installer p.ex. avec le "Package Installer" dans l'application R, ou bien avec la commande:

In [ ]:
install.packages(c("pscl"))

Par la suite, nous pouvons effectuer le calcul avec:

In [12]:
pR2(reglogit)

llh      llhNull           G2     McFadden         r2ML         r2CU 
-466.9194300 -905.1144107  876.3899615    0.4841321    0.4737842    0.6450351

ce qui nous donne un McFadden $R^2$ de 0.484.

Une autre alternative pour évaluer la qualité du modèle est la **matrice de confusion**. Dans cette matrice, nous pouvons voir le nombre de régions qui sont prédites juste ou faux:

![](figures/confusion-matrix.png)

Pour calculer la matrice de confusion, nous avons besoin de deux étapes. Faire attention aux paramètres `reglogit` et `d$typologie_binaire` qui sont les deux seuls à changer:

In [13]:
predict = predict(reglogit, type = 'response')
table(d$typologie_binaire, predict > 0.5)

   
    FALSE TRUE
  N   381  135
  U    69  780

Et avec cela il est facile de calculer la proportion de valeur correctement prédites:

In [14]:
(381+780) / (381+135+69+780)

[1] 0.8505495

ou autrement dit la prédiction est correcte pour 85% des cas.

D'autres méthodes d'évaluation existent, et nous allons continuer cette discussion ultérieurement. L'évaluation de la qualité d'un modèle est en effet un problème important qui mérite de l'attention.

En plus de l'évaluation du modèle au complet, nous devons encore évaluer la contribution au modèle de chacun des coefficient. Pour faire cela, nous pouvons faire une analyse de variance de la déviance des coefficients:

In [15]:
anova(reglogit, test="Chisq")

,Df,Deviance,Resid. Df,Resid. Dev,Pr(>Chi)
NULL,NA,NA,1364,1810.2288,NA
ADO,1,275.309145,1363,1534.9197,7.903966e-62
NADHO,1,85.962594,1362,1448.9571,1.833716e-20
NADRET,1,21.410042,1361,1427.5470,3.708238e-06
AD3PRIM,1,492.027907,1360,935.5191,5.158988e-109
AD3SEC,1,1.680273,1359,933.8389,1.948882e-01


La colonne *«Resid. Dev»* nous indique la déviance résiduelle du modèle tenant compte uniquement des coefficient sur la même ligne et ceux en dessus. Au départ, sans coefficient, la déviance est de 1810.2, puis descend à 1534.9 avec l'ajout de la variable `ADO`, à 1448.96 avec la variable `NADHO`, ainsi de suite. La différence avec la valeur du dessus donne une indication de la contribution du coefficient au modèle.

Dans notre cas, la variable `AD3PRIM` contribue le plus à notre modèle (diminution de 1427.5 à 935.5).

La différence dans cette déviance résiduelle est indiquée dans la colonne ***«Deviance»***.